![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `meta-llama/llama-3-2-11b-vision-instruct` to run as an AI service

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for watsonx.ai AI service.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The learning goal for your notebook is to leverage AI services to generate accurate and contextually relevant responses based on a given image and a related question.


## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Create AI service](#ai_service)
- [Testing AI service's function locally](#testing)
- [Deploy AI service](#deploy)
- [Example of Executing an AI service](#example)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependencies

In [ ]:
!pip install -U "ibm_watsonx_ai>=1.1.22" | tail -n 1

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=wx) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select watsonx.ai Runtime instance and press **Create**
- Go to **Manage** tab
- Copy `Space GUID` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

In [3]:
import os

try:
    space_id = os.environ["SPACE_ID"]
except KeyError:
    space_id = input("Please enter your space_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [7]:
from ibm_watsonx_ai import APIClient

api_client = APIClient(credentials=credentials, space_id=space_id)

Specify the `model_id` of the model you will use for the chat with image.

In [8]:
model_id = "meta-llama/llama-3-2-11b-vision-instruct"

<a id="ai_service"></a>
## Create AI service

Prepare function which will be deployed using AI service.

Please specify the default parameters that will be passed to the function.

In [9]:
def deployable_ai_service(context, space_id=space_id, url=credentials["url"], model_id=model_id, params={"temperature": 1}, **kwargs):

    import requests
    import base64
    from ibm_watsonx_ai import APIClient, Credentials
    from ibm_watsonx_ai.foundation_models import ModelInference

    api_client = APIClient(
        credentials=Credentials(url=url, token=context.generate_token()),
        space_id=space_id,
    )

    model = ModelInference(
        model_id=model_id,
        api_client=api_client,
        params=params,
    )

    def generate(context) -> dict:

        api_client.set_token(context.get_token())

        payload = context.get_json()
        question = payload["question"]
        image_url = payload["image_url"]

        response = requests.get(image_url)
        response.raise_for_status()
        base64_image = base64.b64encode(response.content).decode('utf-8')

        messages = [
            {
                "role": "user",
                "content": [
                {
                    "type": "text",
                    "text": question
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "data:image/jpeg;base64," + base64_image,
                        "detail": "auto"
                    }
                }
                ]
            }
        ]


        response = model.chat(messages=messages)

        return {
            "body": response
            }

    def generate_stream(context):

        api_client.set_token(context.get_token())

        payload = context.get_json()
        question = payload["question"]
        image_url = payload["image_url"]

        response = requests.get(image_url)
        response.raise_for_status()
        base64_image = base64.b64encode(response.content).decode('utf-8')

        messages = [
            {
                "role": "user",
                "content": [
                {
                    "type": "text",
                    "text": question
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "data:image/jpeg;base64," + base64_image,
                        "detail": "auto"
                    }
                }
                ]
            }
        ]

        yield from model.chat_stream(messages)

    return generate, generate_stream

<a id="testing"></a>
## Testing AI service's function locally

You can test AI service's function locally. Initialise `RuntimeContext` firstly.

In [10]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=api_client)

In [26]:
local_function = deployable_ai_service(context=context)

Please retrieve an image and display it. This example is based on the IBM logo.

In [27]:
import requests
from IPython.display import Image

image_url = "https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/logo/ibm_logo.jpg"

response = requests.get(image_url)

Image(url=image_url, width=600)

Prepare request json payload for local invoke.

In [28]:
context.request_payload_json = {"question": "Describe the image", "image_url": image_url}

Execute the `generate` function locally.

In [29]:
resp = local_function[0](context)
resp

{'body': {'id': 'chat-ba0d11524b474e8fa66dd48b951b2905',
  'model_id': 'meta-llama/llama-3-2-11b-vision-instruct',
  'model': 'meta-llama/llama-3-2-11b-vision-instruct',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': 'The image displays the IBM logo, which is a blue graphic of horizontal bars arranged in a specific pattern. \n\n* The IBM logo is centered in the image.\n\t+ The logo is blue and consists of a series of horizontal bars.\n\t+ The bars are arranged in a staggered pattern, with some bars being longer than others.\n\t+ The bars are a deep blue color, which is the primary color of the IBM brand.\n* There is a small, blue registered trademark symbol located below and to the right of the IBM logo.\n\t+ The symbol is a circle with the letter "R" inside it.\n\t+ It is smaller than the IBM logo but still easily visible.\n* The background of the image is white.\n\t+ The white background provides a clean and simple contrast to the blue IBM logo.\n\t+ 

Execute the `generate_stream` function locally.

In [15]:
response = local_function[1](context)

In [16]:
for chunk in response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

The image displays the IBM logo, which is a stylized representation of the company's name in blue text against a white background. The logo is made up of horizontal lines that form the letters "IBM" in a unique and distinctive way.

* The word "IBM" is written in blue text, with each letter composed of several horizontal lines.
	+ The lines are arranged in a staggered pattern, with some lines being longer or shorter than others.
	+ The lines are also spaced unevenly, giving the text a slightly irregular appearance.
* A small registered trademark symbol is located to the right of the word "IBM".
	+ The symbol is also blue and is centered below the "M" in the word "IBM".
	+ It is a small circle with an "R" inside it, indicating that the image is a registered trademark.
* The background of the image is white, which provides good contrast to the blue text and makes the image stand out.
	+ The white background also helps to make the logo appear more modern and sleek.

Overall, the image eff

<a id="deploy"></a>
## Deploy AI service

Store AI service with previous created custom software specifications.

In [17]:
sw_spec_id = api_client.software_specifications.get_id_by_name("runtime-24.1-py3.11")
sw_spec_id

'45f12dfe-aa78-5b8d-9f38-0ee223c47309'

In [18]:
meta_props = {
    api_client.repository.AIServiceMetaNames.NAME: "AI service with SDK",
    api_client.repository.AIServiceMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}
stored_ai_service_details = api_client.repository.store_ai_service(deployable_ai_service, meta_props)

In [19]:
ai_service_id = api_client.repository.get_ai_service_id(stored_ai_service_details)
ai_service_id

'8a37cd0e-42f1-4f31-9680-c498fdeda39d'

Create online deployment of AI service.

In [20]:
meta_props = {
    api_client.deployments.ConfigurationMetaNames.NAME: "AI service with SDK",
    api_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

deployment_details = api_client.deployments.create(ai_service_id, meta_props)



######################################################################################

Synchronous deployment creation for id: '8a37cd0e-42f1-4f31-9680-c498fdeda39d' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='7aacf0e9-111b-42f9-899b-8b88405cf7bf'
-----------------------------------------------------------------------------------------------




Obtain the `deployment_id` of the previously created deployment.

In [21]:
deployment_id = api_client.deployments.get_id(deployment_details)

<a id="example"></a>
## Example of Executing an AI service.

Execute `generate` method.

In [22]:
question = "Describe the image"

deployments_results = api_client.deployments.run_ai_service(
    deployment_id, {"question": question, "image_url": image_url}
)

In [23]:
import json

print(json.dumps(deployments_results, indent=2))

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The image is a mockup of the IBM logo, comprising horizontal blue bars of varying lengths arranged in a design resembling stacked letters J and I, respectively. A smaller circle bearing the text \"\u00ae,\" likely to represent a \"Registered Graphic,\" to the right of the outstretched \"M\" signifies that the logo consists of a Registered Graphic. \n\nThe logo evokes a connection between detailing and the mechanical or computational aspects of language represented in a computer program. The blocks of letters J and I being in a digital format indicate how machine-process and letters stay connected; it has captured the roots of the an illiterate language spoken through machine-like and mathematical language. This symbol stretches far further as to romance of a digital culture in the spirit of story-telling. Hence, Barings closes reinforces IBM as one of the top digital cultures 

Execute `generate_stream` method.

In [24]:
question = "Describe the image"

deployments_results = api_client.deployments.run_ai_service_stream(
    deployment_id, {"question": question, "image_url": image_url}
)

In [25]:
import json

for chunk in deployments_results:
    chunk_json = json.loads(chunk)
    if chunk_json["choices"]:
        print(chunk_json["choices"][0]["delta"].get("content", ""), end="", flush=True)

The image displays the IBM logo, which is a blue graphic of the letters "IBM" in a stylized font. The logo is set against a white background and features a registered trademark symbol (R) to the right of the letters.

*   **Logo:**
    *   The logo is centered in the image and consists of the letters "IBM" in a stylized font.
    *   The letters are blue and have a series of horizontal lines extending from the top and bottom of each letter.
    *   The lines are evenly spaced and create a sense of depth and dimensionality.
*   **Background:**
    *   The background of the image is white, which provides a clean and neutral contrast to the blue logo.
*   **Registered Trademark Symbol:**
    *   To the right of the logo, there is a registered trademark symbol (R) in blue.
    *   The symbol is small and unobtrusive, but it serves as a reminder that the logo is a registered trademark.

Overall, the image effectively communicates the IBM brand identity through its use of a distinctive logo 

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to create and deploy AI service using `ibm_watsonx_ai` SDK.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Mateusz Szewczyk**, Software Engineer at watsonx.ai.

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.